In [ ]:
!pip install --upgrade --user google-cloud-aiplatform google-cloud-storage langchain pypdf ratelimit backoff langchain-google-vertexai langchain-openai google-cloud-bigquery

In [ ]:
# get project ID
PROJECT_ID = ! gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1" # @param {type:"string"}

# define project information manually if the above code didn't work
if PROJECT_ID == "(unset)":
  PROJECT_ID = "[your-project-id]" # @param {type:"string"}

print(PROJECT_ID)

## Initial Vertex AI

In [ ]:
# init the aiplatform package
from google.cloud import aiplatform
aiplatform.init(project=PROJECT_ID, location=LOCATION)

## Helper funtion to Generate Embeddings

In [ ]:
from vertexai.language_models import TextEmbeddingModel

def text_embedding(question) -> list:
    from vertexai.language_models import TextEmbeddingInput
    """Text embedding with a Large Language Model."""
    model = TextEmbeddingModel.from_pretrained("textembedding-gecko@002")

    question_embedding = TextEmbeddingInput(
        text=question,
        task_type="RETRIEVAL_QUERY"
    )

    embeddings = model.get_embeddings([question_embedding])
    for embedding in embeddings:
        vector = embedding.values
    return vector

In [ ]:
emb1 = text_embedding("Hello World")
print(emb1)

## Helper Functions to answer questions using Gemini and PaLM LLMS


In [ ]:
from google.cloud.aiplatform_v1beta1.types.content import SafetySetting, HarmCategory
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
from vertexai.language_models import TextGenerationModel

def answer_question_gemini(prompt):
  model = GenerativeModel("gemini-pro")
  response = model.generate_content(
    prompt,
    generation_config={
        "max_output_tokens": 8192,
        "temperature": 0.5,
        "top_p": 0.5,
        "top_k": 10,
    },
  stream=False,
  )
  try:
    return response.text
  except:
    print("An Error Ocuured Creaning the Data")
    return "An Error Ocuured Creaning the Data"


def answer_question_palm(prompt):
  parameters = {
        "candidate_count": 1,
        "max_output_tokens": 1024,
        "temperature": 0.9,
        "top_p": 1
    }

  model = TextGenerationModel.from_pretrained("text-bison")
  response = model.predict(
        prompt,
        **parameters
  )
  try:
    return response.text
  except:
    print("An Error Ocuured Creaning the Data")
    return "An Error Ocuured Creaning the Data"

## Just for fun add helper function to use GPT

In [ ]:
import getpass
import os

# getpass will prompt for an API Key
API_KEY = getpass.getpass("Provide your OpenAI API Key")

In [ ]:
from langchain_openai import OpenAI

def answer_question_gpt(prompt):
  llm = OpenAI(openai_api_key=API_KEY)
  response = llm.predict(prompt)
  return response


In [ ]:
print(answer_question_gpt("Tell me a joke"))

## Hepler function to use VECTOR_SEARCH funtion to retrieve data from BQ

In [ ]:
def get_closest_pages_from_biguery(question_embedding, num_results=5):
    from google.cloud import bigquery

    client = bigquery.Client()

    sql = """
    SELECT base.cleaned_page, distance
    FROM
      VECTOR_SEARCH(
        TABLE embeddings_ds.embeddings_data,
        'embedding',
        (SELECT {0} as embedding),
        top_k => {1}
        );
    """.format(question_embedding, num_results)

    query_job = client.query(sql)
    data = ""
    for row in query_job:
        data += row.cleaned_page + "\n"
        #print("Distance: {0} - {1}".format(row.distance, row.cleaned_page[:50]))
    return data

## Helper Function to Build the Prompt for the LLM

In [ ]:
def build_prompt(data, question):
    prompt = """
    context: Answer the question using the following Information.

    Information: {0}

    question: {1}

    """.format(data, question)
    return prompt


In [ ]:
def answer_question(question):

  question_embedding = text_embedding(question)
  data = get_closest_pages_from_biguery(question_embedding)
  prompt = build_prompt(data, question)

  answer_gemini = answer_question_gemini(prompt)
  answer_palm = answer_question_palm(prompt)
  answer_gpt = answer_question_gpt(prompt)

  return answer_gemini, answer_palm, answer_gpt

In [ ]:
from IPython.core.display import display, HTML


QUESTION = "What is the minimum safe cooking temperature for chicken?"
QUESTION = "What is the temperature danger zone?"
QUESTION = "What are good food handling practices?"
QUESTION = "What are best practices for handling fish and seafood?"
QUESTION = "How should you store refrigerated food?"

answer_gemini, answer_palm, answer_gpt = answer_question(QUESTION)


print(QUESTION)
print("--------------------------------")
display("Gemini answer: {0}".format(answer_gemini))
print("--------------------------------")
display("PaLM answer: {0}".format(answer_palm))
print("--------------------------------")
display("GPT answer: {0}".format(answer_gpt))
